# CIS 1902 Homework 5: Deep Learning

Welcome to Homework 5! This homework will be focused on deep learning. In this homework, you will be using a variety of tools and libraries that you have learned in lecture to build deep learning models, that actually solve real-world problems. Deep learning has tons of applications, ranging from voice recognition to image recognition to natural language processing.


The foundation of deep learning is the neural network and the convolutional neural network (CNN). The neural network is a series of layers that are connected to one another, which borrows from the structure of the human brain. The CNN is a type of neural network that is particularly good at image recognition, and at a high-level is a parametric model with usually a huge number of parameters.

While the neural network model is comprehensible to us, it introduces a new set of terminologies. Utilizing this model, we'll establish its architecture, including input, hidden, and output layers, and apply it to various real-world problems.

Given that deep learning's primary successes lie in image recognition, language processing, and similar fields, we'll also delve into Convolutional Neural Networks (CNNs), specialized for image processing. We'll compare its performance to traditional neural networks.


## Part 1: Importing Libraries

Let's start by importing the libraries that we will be using in this homework.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.model_selection import train_test_split
import torch
import torchvision
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from collections import Counter
from PIL import Image
from skimage import io, transform
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import math

Next, let's setup GPU capabilities for our notebook. The cell below sets up a CUDA device to use with torch, if it is available. If you are using Google Colab, you can enable GPU acceleration by going to `Runtime` -> `Change runtime type` and selecting `GPU` as the hardware accelerator.

Note that you can create a PyTorch tensor T that resides on the GPU using the command T = torch.Tensor(..., device=cuda). You can also copy existing tensors to the GPU using the to method, e.g. T.to(device) (make sure to overwrite T with the result of the function call). If you do not have a GPU, these commands will not work and you will need to use the CPU.


In [ ]:
torch.manual_seed(42)  # For grading consistency
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(device)

In [ ]:
device

In [ ]:
# Test Case #1
# check that device is set to cuda if available
if torch.cuda.is_available():
    if str(device) == "cuda:0":
        print("Test Case 1 Passed")
    else:
        print("Test Case 1 Failed")
else:
    print("Test Case 1 Passed")

The real-world problem we will be solving in this homework is the classification of images, or more specifically, the classification of handwritten digits. We will be using the MNIST dataset, which is a dataset of 28x28 grayscale images of handwritten digits (0-9). The dataset contains 60,000 training images and 10,000 testing images. The goal is to build a model that can accurately classify these images into their respective digits.

We will be using the dataset to train 3 models and compare their performance on the test set. The models we will be using are:

1. Logistic Regression
2. Neural Network
3. Convolutional Neural Network


<div>
<img src="https://s2.loli.net/2023/03/26/GwFJhNeskzE5Ptx.webp", width="800"/>
</div>


## Part 2: Data Preprocessing (PyTorch Dataset and DataLoader)

To ensure consistency in data loading across various models, PyTorch offers two essential components: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. These primitives facilitate the utilization of pre-loaded datasets and custom data.

The `Dataset` module manages samples along with their respective labels, while `DataLoader` encapsulates an iterable structure around the `Dataset`, simplifying sample access. Additional details and practical examples of these components can be explored in the PyTorch documentation.

While PyTorch furnishes `Dataset` and `DataLoader` for several popular datasets, custom `Dataset` creation might be necessary for specific data processing requirements. PyTorch also provides a catalog of built-in datasets, which can be accessed through the official documentation.

The `Dataset` class encompasses three key components:

- `__init__`: This method is responsible for setting up the parameters used within the class, such as transforms, which corresponds to the applied transformations.
- `__len__`: Its purpose is to ensure that `len(dataset)` returns the dataset's size.
- `__getitem__`: This function supports indexing, allowing `dataset[i]` to retrieve the i-th sample (in the case of images, this would typically be an image-label pair).

Custom `Dataset` creation involves overriding the aforementioned functions.

Documentation on creating custom datasets can be accessed here.

Fortunately, for classic image datasets like MNIST, the need for creating custom `Dataset` and `DataLoader` objects is eliminated, as PyTorch includes them by default upon import.


**TODO**: Implement the following sequence of transformations using `transforms.Compose`, and assign the resulting transformation pipeline to the variable `transform`:

1.  **Resize**: Adjust the dimensions of each image in the dataset to 28x28 pixels, ensuring a consistent size across all images.
2.  **ToTensor**: Convert the images into PyTorch tensors, facilitating their use within the framework.
3.  **RandomRotation**: Apply a random rotation to each image within a range of -45° to 45°, introducing variability to the dataset.
4.  **RandomPerspective**: Introduce a random perspective transformation with a distortion scale of 0.3 to approximately half of the images (50% probability).
5.  **Normalize**: Standardize the tensor images using a mean of 0.1307 and a standard deviation of 0.3081, adjusting pixel values for more effective model training.

---

**NOTE**: For a comprehensive understanding of how each transformation works and to see examples in action, it is strongly advised to consult the [PyTorch documentation on transformations](https://pytorch.org/vision/stable/transforms.html) and explore [visual illustrations of transformations](https://pytorch.org/vision/main/auto_examples/transforms/plot_transforms_illustrations.html) provided by PyTorch.


In [ ]:
# TODO: Define the transform

transform = 

Now that we have defined the transforms we are going to apply to the dataset, let's instantiate `Dataset` objects for both the training and testing sets using predefined PyTorch Datasets. You can refer to the documentation [here](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html).

Considering the whole dataset is quite large for running the model efficiently, we will also downsample by reducing the size of the dataset by half.

---

**TODO**:

- Initiate the `Dataset` object for the training set as `train_dataset`.
- Downsample the training dataset by getting the data at even-numbered indices, specified as `train_subset`.
- Initiate the `Dataset` object for the testing set as `test_dataset`.
- Downsample the testing dataset by getting the data at even-numbered indices, specified as `test_subset`.

---

**Hint**: Check out [`torch.utils.data.Subset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset) for downsampling the dataset.

**Note**: When initiating the `Dataset` object for both the training and testing sets, in addition to other arguments, please set these arguments as follows:

- `root = './data'`
- `download = True`


In [ ]:
# TODO: Initiate the Dataset object for the training set as train_dataset
train_dataset =

# TODO: Downsample training by getting the train data at even-numbered indices, specified as train_subset
train_subset =

# TODO: Initiate the dataset object for the test set as test_dataset
test_dataset =

# TODO: Downsample test by getting the test data at even-numbered indices, specified as test_subset
test_subset =

With `trainDataset` and `testDataset` prepared, it's time to set up dataloaders for both datasets.

Utilize the `DataLoader` class to encapsulate the datasets into dataloaders. For more detailed instructions, consult the [DataLoader documentation](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

---

**TODO**:

- Create a `DataLoader` instance for `train_subset` named `train_loader`.
- Create a `DataLoader` instance for `test_subset` named `test_loader`.

---

**NOTE**: When initializing the `DataLoader` instances, ensure to configure the `batch_size` to match the value of the `batch` variable previously established. Also, enable `shuffle` by setting it to `True` and set `num_workers` to 0.


In [ ]:
batch = 64

# TODO: Initiate the DataLoader object for the training set as train_loader
train_loader =

# TODO: Initiate the DataLoader object for the test set as test_loader
test_loader =

## Part 3: Data Summary and Visualization

Now that we have our dataset, let's take some time to understand the data we are working with. We will visualize some of the images in the dataset and understand the distribution of the labels in the dataset.

**TODO**

- **Train**

  - Determine the count of distinct labels present in the training dataset and assign this value to `train_num_labels`.
  - Instantiate a `DataLoader` named `train_loader_bar_plot` using `train_subset`, configured with:
    - `batch_size` set to the total number of items in `train_subset`
    - `shuffle` enabled (`True`)
    - `num_workers` set to 0
  - Construct a dictionary named `train_subset_dict` where each key represents a label, and the corresponding value is the count of images under that label.

- **Test**

  - Identify the total number of unique labels in the testing dataset and store this figure in `test_num_labels`.
  - Establish a `DataLoader` named `test_loader_bar_plot` for `test_subset`, with the settings:
    - `batch_size` equal to the size of `test_subset`
    - `shuffle` activated (`True`)
    - `num_workers` as 0
  - Create a dictionary `test_subset_dict` mapping each label to the quantity of images associated with that label.


In [ ]:
train_num_labels = 

test_num_labels =

train_loader_bar_plot = 

test_loader_bar_plot =

train_subset_dict =

test_subset_dict = 

Now we are going to use the matplotlib skills we worked on in the last homework, to visualize the data.

**TODO:**

- Construct a bar plot to visualize the frequency distribution of labels in the training dataset. Use `train_subset_dict` for data, set the x-axis label to `Labels`, the y-axis label to `Frequency`, and the plot title to `Distribution of Training Set Labels`.
- Similarly, generate a bar plot for the label frequency distribution in the testing dataset using `test_subset_dict`. Label the x-axis as `Labels`, the y-axis as `Frequency`, and title the plot `Distribution of Testing Set Labels`.
- Ensure each bar displays its label count above it (for example, a bar representing label 1 with 1200 instances should have "1200" displayed above).
- To keep the file size manageable for submission, adjust the figure size to (8,6).

---

**NOTE:**

- The y-axis represents the total number of instances per label.
- The x-axis should denote the numerical value of the labels found in the MNIST dataset.


In [ ]:
# Train Subset Bar Plot

In [ ]:
# Test Subset Bar Plot

Now, let's visualize some of the images in the dataset. Take some arbitrary indices, and print out 6 of the images in the training set.

**TODO**

- Iterate over `train_dataset` using a list of arbitrary indices (length: 6) with a for-loop.
- Within each iteration of this loop:
  - Display the image at the current index from `train_dataset` as a grayscale image. Use a 2 by 3 subplot arrangement for plotting, leveraging matplotlib.
  - Use `plt.title` to label each subplot with the corresponding image label from the dataset.
  - Ensure that grid lines and axis labels are not visible in the plots.
- To keep the file size suitable for submission, set the total figure size to (8,4). Note that this size refers to the overall figure, not to individual subplots.


In [ ]:
# TODO: Visualize 6 arbitrary images from the train_subset dataset

## Part 4: Logistic Regression

We will first start by building a logistic regression model to classify the images. We have seen Logistic Regression before, but this time we will use PyTorch to build the model.


In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: initialize the neural network layers (flatten, and then sequential with a Linear Layer(28 * 28, 10), and a sigmoid activation)
    
    def forward(self, x):
        # TODO: define the forward pass and return outputs (call the layers in the init function in sequence here)
        outputs = 
        return outputs

**TODOs:**

1.  **Instantiate Logistic Regression:**

    - Create a logistic regression model and assign it to the variable `logreg`. Ensure that the model is moved to the GPU device for optimized computation.

2.  **Set Loss Criterion:**

    - Define the loss criterion as `CrossEntropyLoss`. For detailed information, consult the PyTorch [documentation on loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions). Remember, `CrossEntropyLoss` inherently applies softmax, so there's no need to include an additional softmax layer in your model.

3.  **Complete the Training Loop:**

    - Fill in the missing sections in the training loop as indicated by the `#TODO` comments. This involves correctly updating the model weights through backpropagation and managing gradients.

4.  **Track Training Accuracy:**

    - For each training epoch, record the accuracy into a list named `acc_LIST_log`. Make sure the recorded values are of type float.

5.  **Track Average Loss:**

    - Similarly, log the average loss per epoch into a list called `loss_LIST_log`, ensuring the data type is float.

**Additional Instructions:**

- Use the `train_loader` for training the logistic regression model.
- The optimizer should be Adam with a learning rate of 1e-4.
- **Important:** If you observe an increase in loss during training, review the model implementation for potential errors. Accurate implementation should not result in loss increments.

**Note:** Ensure that both `acc_LIST_log` and `loss_LIST_log` contain data as floats, not tensors.


In [ ]:
%%time
# TODO: Instantiate the model
logreg = 

# TODO: Set the loss criterion to be Cross Entropy Loss
criterion =

# TODO: Set the optimizer to be Adam with a learning rate of 1e-4
optimizer =

epoch = 10

loss_LIST_log = []
acc_LIST_log = []

# Train the Model
for epoch in range(epoch):
  running_loss = 0.0
  correct = 0
  total = 0
  for inputs, labels in train_loader:
      labels = labels.type(torch.LongTensor) # Cast to Float
      inputs, labels = inputs.to(device), labels.to(device)
      
      ## TODO
      # Step 1: Reset the optimizer tensor gradient every mini-batch
      
      
      # Step 2: Feed the network the train data

      
      # Step 3: Get the prediction using argmax

      
      # Step 4: Find average loss for one mini-batch of inputs

      
      # Step 5: Do a back propagation

      
      # Step 6: Update the weight using the gradients from back propagation by learning step

      # Step 7: Get loss and add to accumulated loss for each epoch

      # Step 8: Get number of correct prediction and increment the number of correct and total predictions after this batch
      # Hint: we need to detach the numbers from GPU to CPU, which stores accuracy and loss


      
  # Step 9: Calculate training accuracy for each epoch (should multiply by 100 to get percentage), store in variable called 'accuracy', and add to acc_LIST_log

  
  # Step 10: Get average loss for each epoch and add to loss_LIST_log
  
  # Step 11: Print the loss and accuracy for each epoch

Use your matplotlib skills to visualize the training process. Plot the training accuracy and loss over the epochs.


Run the trained logistic regression model on the test set and report the accuracy.


In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor) 
        images, labels = images.to(device), labels.to(device)
        # TODO: Get the outputs by passing the test data through the model
        outputs = 
       
        # TODO: Get the prediction using argmax
        _, predicted = 
        
        # TODO: Get number of correct prediction and add to correct and total
        correct +=
        total += 

# TODO: Calculate test accuracy for logistic regression (should multiply by 100)
test_acc_log = 

# TODO: Print the test accuracy


## Part 5: Neural Network

Now that you have built the logistic regression model, you probably noticed that the accuracy is not very high. This is because logistic regression is a linear model, and is not very good at capturing the complex relationships in this sort of problem setting. Let's try a conventional neural network model to see if we can improve the accuracy (feedforward neural network).

We will create a feedforward neural network (FNN) with 1 hidden layer with 256 output units (note that the last layer should have 10 output units, corresponding to the 10 classes).

A key question to consider is the choice of activation function. Feel free to play around with different activation functions and see how they affect the model's performance. Follow the slides and add activations to each of the hidden layers.


In [ ]:
class FNN(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: initialize the neural network layers

    def forward(self, x):
        # TODO: define the forward pass and return outputs
        outputs =
        return outputs

**TODOs**

1.  **Initialize the FNN Model:**

    - Assign your Feedforward Neural Network (FNN) model to the variable `fnn`. Ensure the model is transferred to the appropriate device environment for efficient computation.

2.  **Define the Loss Function:**

    - Use `CrossEntropyLoss` as the loss function for this model. Consult the [PyTorch documentation on loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions) for more information. This function is suitable for classification tasks and automatically applies the softmax layer.

3.  **Complete Training Loop:**

    - Address the incomplete sections within the training loop as indicated. Properly manage weight updates and gradients during backpropagation.

4.  **Record Training Accuracy:**

    - Store the accuracy of each epoch in a list named `acc_LIST_FNN`. Ensure that these values are stored as floats.

5.  **Log Average Loss:**

    - Similarly, keep track of the average loss per epoch in a list named `loss_LIST_FNN`, ensuring the values are in float format.

**Guidelines:**

- The optimizer for this task is predefined as Adam, with a learning rate of 1e-4.
- To properly handle gradients, invoke `optimizer.zero_grad()` to reset gradients before each `.backward()` operation.
- **Caution:** An increase in loss during training indicates a potential issue with the model's implementation. It's crucial to review and correct any errors in such cases.

**Execution Time:**

- The execution of the following code block is estimated to take approximately 4-5 minutes.

**Note:** Both `acc_LIST_FNN` and `loss_LIST_FNN` must only contain values of type float, not tensors.


In [ ]:
%%time
# Sending the data to device (CPU or GPU)
# TODO instantiate the FNN model to variable fnn
fnn = 

# Set the loss criterion as CrossEntropyLoss
criterion = 

# Set the optimizer as Adam with a learning rate of 1e-4
optimizer = 

epoch = 10

acc_LIST_FNN = []
loss_LIST_FNN = []
# Train the FNN
for epoch in range(epoch):
  running_loss = 0.0
  correct = 0
  total = 0
  for inputs, labels in train_loader:
      labels = labels.type(torch.LongTensor) # Cast to Long
      inputs, labels = inputs.to(device), labels.to(device)
      
      ## TODO
      # Step 1: Reset the optimizer tensor gradient every mini-batch
      
      # Step 2: Feed the network the train data
      
      # Step 3: Get the prediction using argmax
      
      # Step 4: Find average loss for one mini-batch of inputs
      
      # Step 5: Do a back propagation
      
      # Step 6: Update the weight using the gradients from back propagation by learning step

      # Step 7: Get loss and add to accumulated loss for each epoch

      # Step 8: Get number of correct prediction and increment the number of correct and total predictions after this batch

  # Step 9: Calculate training accuracy for each epoch (should multiply by 100 to get percentage), store in variable called 'accuracy', and add to acc_LIST_FNN
  
  # Step 10: Get average loss for each epoch and add to loss_LIST_FNN

  
  # Step 11: Print the loss and accuracy for each epoch

Use your matplotlib skills to visualize the training process. Plot the training accuracy and loss over the epochs.


Now, run the trained FNN model on the test set and report the accuracy.


In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor) 
        images, labels = images.to(device), labels.to(device)

        # TODO: Get the outputs by passing the test data through the model
        outputs =

        # TODO: Get the prediction using argmax
        _, predicted =

        # TODO: Get number of correct prediction and add to correct and total
        correct +=
        total +=

# TODO: Calculate test accuracy for FNN (should multiply by 100)
test_acc_FNN =

# Print the test accuracy


## Part 6: Convolutional Neural Network [EC (Optional)]

Finally, we will build a convolutional neural network (CNN) to classify the images. CNNs are particularly good at image recognition tasks, and we should expect to see a significant improvement in accuracy compared to the logistic regression and feedforward neural network models.

In this task, you will design a Convolutional Neural Network (CNN) architecture using PyTorch. The key components of CNNs include:

- Convolutional Layers
- Pooling Layers
- Linear (Fully Connected) Layers
- Activation Functions

**CNN Architecture Requirements:**

Your CNN model should be structured into one or more blocks, with each block comprising the following sequence:

1.  **Convolutional Layer:** This layer applies a set of learnable filters to the input image, capturing spatial features.
2.  **Activation Function:** Following the convolutional layer, an activation function introduces non-linearity to the model, enabling it to learn more complex patterns. Common choices include ReLU (Rectified Linear Unit).
3.  **Max Pooling:** A pooling layer reduces the dimensionality of the data, enhancing the detection of features by summarizing the presence of features in patches of the feature map.

After passing through one or more of these blocks, the output should be:

4.  **Flattened:** The multi-dimensional output from the convolutional blocks is flattened into a single vector to serve as input for the dense layers.
5.  **Linear Layers:** One or more fully connected (dense) layers follow, where each layer is followed by an activation function, except the final layer.
6.  **Output Layer:** The last dense layer's output shape must match the number of classes in your dataset. This layer typically does not have an activation function if you're using a loss function like `CrossEntropyLoss`, which incorporates softmax.

**Design Considerations:**

- For the purposes of this assignment, limit the architecture to no more than two or three convolutional blocks to prevent overly complex models.
- Ensure the final output layer's size corresponds precisely to the number of classes for accurate classification.


To calculate the dimensions of the feature maps at various stages of a Convolutional Neural Network (CNN), especially when dealing with square images, we can create a function that computes the size of the output feature map given the input size, filter size, stride, and padding applied in both convolutional and pooling layers. This calculation is crucial for determining the correct input size for the fully connected layers that follow the convolutional layers in the network.

Here's a generalized function in Python that computes the output dimension of a feature map given these parameters:


In [ ]:
def calculate_output_dimension(input_size, filter_size, stride, padding):
    """
    Calculate the output dimension of a feature map.

    Args:
    - input_size (int): The height/width of the input feature map (assuming a square).
    - filter_size (int): The height/width of the filter (kernel) used (assuming a square).
    - stride (int): The stride with which the filter is applied.
    - padding (int): The amount of padding applied to the input feature map.

    Returns:
    - int: The height/width of the output feature map (assuming a square).
    """
    output_size = math.floor((input_size - filter_size + 2 * padding) / stride) + 1
    return int(output_size)


# Example usage
input_size = 32  # Example input dimension
filter_size = 3  # Example filter size
stride = 1  # Example stride
padding = 0  # Example padding

# Calculate output dimension
output_dimension = calculate_output_dimension(input_size, filter_size, stride, padding)
print(f"Output dimension: {output_dimension}x{output_dimension}")

This function assumes the images are square, meaning the height and width are equal. This simplification is common in many CNN architectures, especially when working with datasets of standardized image sizes (e.g., CIFAR-10, MNIST).

To apply this function, simply plug in the specific values for each convolutional or pooling layer in your network to understand the dimensions at each stage. This is crucial for designing the network architecture, particularly for specifying the input size to the first fully connected layer after the convolutional layers.


**TODOs:**

In your `__init__` method:

1.  Construct between one to three `convolutional blocks`, each comprising a convolution layer, an activation function, and a pooling layer.
2.  `Flatten` the output from the convolutional stages into a tensor.
3.  Set up between one to three `fully-connected layers`.

In your `forward` method (`def forward(self, x)`): 4. Implement the `forward` pass using the layers defined in the `__init__` method.

---

- **Note:** While the specifics of the convolution layers may differ, you might opt to utilize identical pooling layers and activation functions across all blocks. Should you choose uniform parameters for these components, a single instantiation of the pooling layer and the activation function is sufficient.
- **Hint 1:** Leverage the `calculate_output_dimension` function along with the comments on input_dim and output_dim to monitor the dimensions flowing through each layer.
- **Hint 2:** The dimensions you calculate are crucial for determining the input size for the initial fully-connected layer.
- **Hint 3:** Each input to your network is a 28x28 image with a single color channel (gray-scale).
- **Hint 4:** Ensure that the output size of the final fully-connected layer matches the number of classes in your dataset.


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: initialize the neural network layers

    def forward(self, x):
        # TODO

        # Step 1. Pass the images (x) through convolution block 1 and (block 2, 3 if you have built them)

        # Step 2. Flatten the image

        # Step 3. Pass the output through the fully-connected layers (remember to include activation function(s))

        # END TODO
        return outputs

**TODOs**

1.  **Initialize the CNN Model:**

    - Assign your Convolutional Neural Network model to the variable `cnn` and ensure it's moved to the appropriate computational device.

2.  **Define the Loss Function:**

    - Use `CrossEntropyLoss` as your criterion for model evaluation. For further details, refer to the [PyTorch documentation on loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions).

3.  **Complete the Training Loop:**

    - Address the incomplete sections within the training loop to ensure proper model training.

4.  **Compute Training Accuracy:**

    - For each epoch, calculate the training accuracy. Convert the accuracy to percentage form and store these values in a variable named `accuracy`.

5.  **Log Training Accuracy:**

    - Record the training accuracy for every epoch in a list named `acc_LIST_CNN`.

6.  **Log Average Loss:**

    - Similarly, maintain a record of the average loss per epoch in a list called `loss_LIST_CNN`.

**Fixed Parameters:**

- **Optimizer:** The model uses Adam as its optimizer.
- **Epochs:** Training will proceed for 10 epochs.

---

**Hints and Notes:**

- **Weight Updates:** To facilitate proper backpropagation, remember to clear the gradients before each backward pass by calling `optimizer.zero_grad()`.
- **Model Implementation Check:** If you notice an increase in loss during training, reassess your model's implementation for potential errors.
- **Training Duration:** Typically, training should be completed within 5 to 10 minutes.
- **Data Types:** Ensure that `acc_LIST_CNN` and `loss_LIST_CNN` contain values as floats, not tensors.


In [ ]:
%%time
# Sending the data to device (CPU or GPU)
# TODO: instantiate the CNN model to variable cnn
cnn = 

# TODO: set the loss criterion as CrossEntropyLoss
criterion = 

# TODO: set the optimizer as Adam with a learning rate of 1e-4
optimizer = 

epoch = 10

acc_LIST_CNN = []
loss_LIST_CNN = []

# Train the CNN
for epoch in range(epoch):
  running_loss = 0.0
  correct = 0
  total = 0
  for inputs, labels in train_loader:
      labels = labels.type(torch.LongTensor) # Cast to Float
      inputs, labels = inputs.to(device), labels.to(device)
      ## TODO 
      # Step 1: Reset the optimizer tensor gradient every mini-batch
      
      # Step 2: Feed the network the train data
      outputs = 
      
      # Step 3: Get the prediction using argmax
      _, predicted = 
      
      # Step 4: Find average loss for one mini-batch of inputs
      loss = 
      
      # Step 5: Do a back propagation

      # Step 6: Update the weight using the gradients from back propagation by learning step

      # Step 7: Get loss and add to accumulated loss for each epoch
      running_loss += 


      # Step 8: Get number of correct prediction and increment the number of correct and total predictions after this batch
      # Hint: we need to detach the numbers from GPU to CPU, which stores accuracy and loss
      correct += 
      total += 

  # Step 9: Calculate training accuracy for each epoch (should multiply by 100 to get percentage), store in variable called 'accuracy', and add to acc_LIST_CNN
  accuracy =

  # Step 10: Get average loss for each epoch and add to loss_LIST_CNN
  
  # Step 11: Print the loss and accuracy for each epoch
  

You know the drill. Use your matplotlib skills to visualize the training process. Plot the training accuracy and loss over the epochs.


Calculate the accuracy of the CNN model on the test set.


In [ ]:
# TODO
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor) # Cast to Float
        images, labels = images.to(device), labels.to(device)
        # TODO
        # Get the output
        outputs = 
        
        # Get the prediction using argmax
        _, predicted = 
        
        # Get number of correct prediction and add to correct and total
        correct += 
        total +=
       

# Calculate test accuracy for CNN (should multiple by 100)
test_acc_CNN = 

# Print the test accuracy

From the test accuracy of the three models, we can see that the CNN model performs the best. This is expected, as CNNs are particularly good at image recognition tasks.

Digit recognition is used in a variety of applications, such as reading postal codes, recognizing handwritten numbers on bank checks, and recognizing numbers on car license plates. The MNIST dataset is a popular dataset for digit recognition, and the models we have built can be used in these applications.


Now, we want to see where the best model (CNN) is making mistakes. We will visualize some of the images that the CNN model is misclassifying. We may wonder, are there certain digits that are more difficult to classify than others? Are there certain images that are more difficult to classify than others? We will explore these questions by visualizing some of the misclassified images. To figure this out we will create a confusion matrix.


In [ ]:
confusion_matrix_df = 

Visualize the confusion matrix using matplotlib.


What do you observe from the confusion matrix? Are there certain digits that are more difficult to classify than others? Are there certain digits that are easily confused with others?


## Part 7: Applying Knowledge in New Contexts [EC (Optional)]

Now that we've done a tutorial like exercise with the MNIST dataset, let's see if you can apply your knowledge to a new setting with a little less hand holding. You'll work with a dataset of Yelp reviews, aiming to predict the sentiment (positive or negative) of each review based on its content. The dataset comprises 100,000 Yelp reviews, each annotated with a binary rating indicating whether the review is positive (1) or negative (0). The reviews have been pre-processed into a term frequency format, retaining 1,072 significant words. Your task is to develop a neural network model that accurately classifies these reviews using PyTorch. This task is a little more open-ended because we want you to be creative in how you design your model. You can use any of the models you have learned about in this course, and you can experiment with different architectures, activation functions, and hyperparameters. We will provide a few hints and ideas/skeleton code to get you started, but feel free to experiment and try different things.


### Data Preparation

**Objective:**

Prepare the dataset for training and validation, ensuring all data is in the appropriate format (matrices or vectors) for neural network training.

**Tasks:**

1.  Load the Yelp review dataset from the provided CSV file, `YELP_tm_freq.csv`. This file contains pre-processed reviews in term frequency format.
2.  Split the dataset into training and validation sets. Reserve 10,000 samples for validation (`data3_val`) and use the remaining 90,000 samples for training (`data3_train`).
3.  Ensure the training and validation data are structured as matrices (features) and vectors (labels).


### Model Design and Training

**Objective:**

Implement and train a fully connected neural network to classify Yelp reviews based on their sentiment.

**Model Architecture:**

- The network should have two hidden layers.
- The first layer should contain 16 neurons, and the second layer should contain 8 neurons. Use ReLU as the activation function for both layers.
- The output layer should use the sigmoid activation function to output the probability of a review being positive.

**Tasks:**

1.  Define the neural network architecture using PyTorch. Use `Sequential` model type and add layers with `Dense`.
2.  Compile the model with the following configurations:
    - Optimizer: RMSprop
    - Loss function: Sparse Categorical Crossentropy
    - Metric: Accuracy
3.  Train the model on the training set. Use a validation split of 15% to monitor the model's performance during training. Train for 20 epochs with a batch size of 512.


### Model Evaluation

**Objective:**

Evaluate the performance of your trained model on the validation set and make predictions on new data samples.

**Tasks:**

1.  Assess the model's performance on the validation set. Report the accuracy and loss.
2.  Predict the sentiment of the first five reviews in the validation set. Display the predicted probabilities and the actual labels.

We will give extra credit for the best performing models on the validation set.
